In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from datetime import datetime

In [39]:
train = pd.read_csv('train.csv',parse_dates=['purchase_date','release_date'])
# We give up the row containing missing values
train.dropna(axis=0, how='any', inplace=True)
train.head(3)

,id,playtime_forever,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews
0,0,0.000000,False,3700.0,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...",2018-07-02,2013-12-10,372.0,96.0
1,1,0.016667,True,0.0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...",2016-11-26,2015-08-12,23.0,0.0
2,2,0.000000,False,5000.0,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...",2018-07-02,2014-01-28,3018.0,663.0


In [40]:
train['days']=train['purchase_date']-train['release_date']

In [41]:
train['whole_reviews']=train['total_negative_reviews']+train['total_positive_reviews']
train['posi_ratio'] = train['total_positive_reviews']/train['whole_reviews']
train = train.fillna({'posi_ratio':0})
train.to_csv('ok.csv')
train.head(1)

,id,playtime_forever,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,days,whole_reviews,posi_ratio
0,0,0.0,False,3700.0,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...",2018-07-02,2013-12-10,372.0,96.0,1665 days,468.0,0.794872


In [42]:
dummies1 = train["genres"].str.get_dummies(",") 
dummies2 = train['categories'].str.get_dummies(',')
# dummies3 = train['tags'].str.get_dummies(',')
# dummies1
# dummies2
dummies1.index.name = 'id'
dummies2.index.name = 'id'
# dummies3.index.name = 'id'

# dummies2
# dupli = dummies1.columns.values.tolist()
# dummies3 = dummies3.drop(columns=dupli)

training_data = pd.merge(dummies1,dummies2,on='id')
# training_data = pd.merge(training_data,dummies3,on = 'id')
train = train.rename(columns={'id':'ID'})
train.index.name = 'id'
# train
training_data = pd.merge(training_data,train,on='id')

In [43]:
# Delete some features that are not very important
training_data = training_data.drop(columns=
                                   ['genres','categories','tags','ID','total_positive_reviews',
                                            'total_negative_reviews','purchase_date','release_date','Early Access',
                                    'Commentary available','SteamVR Collectibles','Gore','Includes Source SDK','Nudity',
                                   'is_free','Simulation','Remote Play on Tablet','Single-player','Local Co-op','Free to Play',
                                    'Includes level editor','MMO','Sports','VR Support','Violent','Casual'])
training_data.head(5)

,Action,Adventure,Animation & Modeling,Audio Production,Design & Illustration,Indie,Massively Multiplayer,RPG,Racing,Sexual Content,...,Steam Cloud,Steam Leaderboards,Steam Trading Cards,Steam Workshop,Valve Anti-Cheat enabled,playtime_forever,price,days,whole_reviews,posi_ratio
id,,,,,,,,,,,,,,,,,,,,,
0,0,1,0,0,0,1,0,0,0,0,...,1,0,1,0,0,0.000000,3700.0,1665 days,468.0,0.794872
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0.016667,0.0,472 days,23.0,1.000000
2,0,1,0,0,0,1,0,0,0,0,...,1,0,1,0,0,0.000000,5000.0,1616 days,3681.0,0.819886
3,1,0,0,0,0,0,0,1,0,0,...,0,0,1,1,0,1.533333,9900.0,2434 days,64824.0,0.973066
4,1,0,0,0,0,1,0,0,0,0,...,1,1,1,0,0,22.333333,4800.0,2043 days,9364.0,0.944148


In [46]:
test = pd.read_csv('test.csv',parse_dates=['purchase_date','release_date'])
# Make up for missing values of test dataset.
test = test.fillna({'purchase_date':datetime(2018,01,01),'total_positive_reviews':0,'total_negative_reviews':0})

In [47]:
# Calculate the days the player purchases after games were released.
test['days']=test['purchase_date']-test['release_date']

# Calculate the whole game reviews and the positive reviews ratio.
test['whole_reviews']=test['total_negative_reviews']+test['total_positive_reviews']
test['posi_ratio'] = test['total_positive_reviews']/test['whole_reviews']
test = test.fillna({'posi_ratio':0})

# Transform some columns to dummie matirxs.
testdummies1 = test["genres"].str.get_dummies(",") 
testdummies2 = test['categories'].str.get_dummies(',')

testdummies1.index.name = 'id'
testdummies2.index.name = 'id'

dupli = testdummies1.columns.values.tolist()


test_data = pd.merge(testdummies1,testdummies2,on='id')
test = test.rename(columns={'id':'ID'})
test.index.name = 'id'

test_data = pd.merge(test_data,test,on='id')

test_data = test_data.drop(columns=['genres','categories','tags','ID','total_positive_reviews','total_negative_reviews','purchase_date','release_date'])
test_data.head(3)

,Action,Adventure,Casual,Early Access,Free to Play,Gore,Indie,Massively Multiplayer,Nudity,RPG,...,Steam Leaderboards,Steam Trading Cards,Steam Workshop,SteamVR Collectibles,VR Support,is_free,price,days,whole_reviews,posi_ratio
id,,,,,,,,,,,,,,,,,,,,,
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,False,3500,2350 days,3729.0,0.699115
1,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,False,11600,452 days,7997.0,0.720520
2,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,False,2100,848 days,820.0,0.837805


In [48]:
# Extract features that involved in both train dataset and test dataset.
train_feature = training_data.columns.values.tolist()
test_feature = test_data.columns.values.tolist()
a = set(training_data)&set(test_feature)
a

{'Action',
 'Adventure',
 'Captions available',
 'Co-op',
 'Cross-Platform Multiplayer',
 'Full controller support',
 'In-App Purchases',
 'Indie',
 'Local Multi-Player',
 'Massively Multiplayer',
 'Multi-player',
 'Online Co-op',
 'Online Multi-Player',
 'Partial Controller Support',
 'RPG',
 'Remote Play on Phone',
 'Remote Play on TV',
 'Shared/Split Screen',
 'Stats',
 'Steam Achievements',
 'Steam Cloud',
 'Steam Leaderboards',
 'Steam Trading Cards',
 'Steam Workshop',
 'Strategy',
 'days',
 'posi_ratio',
 'price',
 'whole_reviews'}

In [49]:
# See the feature number.
a = list(a)
print(len(a))

29


In [50]:
train_new = training_data[a]
# Unify the format of the dataframe in order to get the traing data.
train_new['days'] =train_new['days'].dt.days
train_new['days'] = train_new['days'].astype(int)


/usr/local/Caskroom/miniconda/base/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/Caskroom/miniconda/base/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [52]:
test_new = test_data[a]

test_new['days'] =test_new['days'].dt.days
test_new['days'] = test_new['days'].astype(int)
# test_new.head(3)

/usr/local/Caskroom/miniconda/base/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/Caskroom/miniconda/base/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [53]:
# The infomation for two datasets.
test_new.info()
train_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90 entries, 0 to 89
Data columns (total 29 columns):
Stats                         90 non-null int64
Strategy                      90 non-null int64
Full controller support       90 non-null int64
Local Multi-Player            90 non-null int64
Cross-Platform Multiplayer    90 non-null int64
Captions available            90 non-null int64
Co-op                         90 non-null int64
Steam Workshop                90 non-null int64
whole_reviews                 90 non-null float64
Adventure                     90 non-null int64
RPG                           90 non-null int64
In-App Purchases              90 non-null int64
Massively Multiplayer         90 non-null int64
Remote Play on TV             90 non-null int64
Multi-player                  90 non-null int64
Steam Leaderboards            90 non-null int64
price                         90 non-null int64
Online Co-op                  90 non-null int64
Action                        9

In [16]:
train_new.to_csv('new_train.csv',index=False)
test_new.to_csv('new_test.csv',index=False)

In [54]:
# Transform the dataframe into training data and testing data.
y = training_data['playtime_forever']
x = train_new
YY = y.values
XX = x.values

In [55]:
# Output the features we use to train the model.
features = train_new.columns.values.tolist()
features

['Stats',
 'Strategy',
 'Full controller support',
 'Local Multi-Player',
 'Cross-Platform Multiplayer',
 'Captions available',
 'Co-op',
 'Steam Workshop',
 'whole_reviews',
 'Adventure',
 'RPG',
 'In-App Purchases',
 'Massively Multiplayer',
 'Remote Play on TV',
 'Multi-player',
 'Steam Leaderboards',
 'price',
 'Online Co-op',
 'Action',
 'Online Multi-Player',
 'Shared/Split Screen',
 'Steam Trading Cards',
 'Steam Achievements',
 'Indie',
 'days',
 'posi_ratio',
 'Partial Controller Support',
 'Remote Play on Phone',
 'Steam Cloud']

In [56]:
# Split the training dataset into train set and validation set.
x1,x2,y1,y2 = train_test_split(XX,YY,test_size=0,random_state=0)

In [57]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [29]:
# clf = MLPRegressor(hidden_layer_sizes=(10,),activation='logistic', solver='adam',
#                   learning_rate='adaptive', warm_start=True)
# clf.fit(x1,y1)
# print(np.sqrt(mean_squared_error(y1,clf.predict(x1))))

In [58]:
# Use Random Forest to predict the playtime.
clf = RandomForestRegressor(max_depth=None, random_state=0,
                            n_estimators=100)
 
clf.fit(x1,y1)
print(np.sqrt(mean_squared_error(y1,clf.predict(x1))))

4.361504345120374


In [60]:
# Use KFold to get our average training accuracy.
from sklearn.model_selection import KFold
error = []
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(x1):
    X_train, X_test = x1[train_index], x1[test_index]
    y_train, y_test = y1[train_index], y1[test_index]
#     clf=SVR(C=1e1,max_iter=400)
# #     clf = MLPRegressor(hidden_layer_sizes=(50,),activation='logistic',
# #                         solver='adam')
#     clf = MLPRegressor(hidden_layer_sizes=(100,),activation='logistic',
#                   solver='adam',batch_size=32,
#                   learning_rate='adaptive', warm_start=True)
    
    clf = RandomForestRegressor(max_depth=2, random_state=0,
                            n_estimators=200)
 
    clf.fit(X_train,y_train)

    result = clf.predict(X_test)
#     print(result)
    MSE = mean_squared_error(y_test,result)
    error.append(np.sqrt(MSE))
    print(MSE)
print('The mean error of this model is:',np.mean(np.array(error)))

63.24118134724097
76.7868840840454
113.68510197926301
38.90123795309094
369.75490587690547
('The mean error of this model is:', 10.56873287388834)


In [61]:
# Sort the features according to their importance of the model.
importances=clf.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(x1.shape[1]):
    print("%s" % features[indices[f]])

whole_reviews
days
price
Steam Leaderboards
Remote Play on TV
Steam Workshop
posi_ratio
Stats
Online Co-op
Strategy
RPG
Online Multi-Player
Co-op
Remote Play on Phone
Steam Trading Cards
Adventure
Action
Indie
Full controller support
Multi-player
Steam Achievements
Massively Multiplayer
Captions available
Shared/Split Screen
Steam Cloud
Partial Controller Support
In-App Purchases
Cross-Platform Multiplayer
Local Multi-Player


In [26]:
# np.sqrt(mean_squared_error(y2,clf.predict(x2)))

9.437957836400544

In [62]:
print(test_new.shape,train_new.shape)
testing = test_new.values


((90, 29), (355, 29))


In [63]:
# Get the prediction of the test set.
Y = clf.predict(testing)

In [64]:
# Get the csv file we need for submission.
final = pd.DataFrame(columns=['id','playtime_forever'])
final['playtime_forever']=Y
final['id'] = range(90)
TEST = pd.merge(test,final,on='id')
final.to_csv('xiu2.csv',index=False)

In [65]:
TEST.to_csv('TEST_final.csv')